# Learning about Convolutions

Learning about convolutions step-by-step.

In [1]:
import torch
import torch.nn as nn

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
device

device(type='cuda', index=0)

## 1D Convolutions

Lets start with a 3x4 tensor

In [4]:
x = torch.tensor([[1., -2., 3., -4.],
                  [1., -2., 3., -4.],
                  [1., -2., 3., -4.]])

In [5]:
x.shape

torch.Size([3, 4])

For `Conv1d()`, `in_channels` has to equal 3 since we have 3 rows

In [6]:
conv = torch.nn.Conv1d(in_channels = 3, out_channels = 3, kernel_size = 1)
conv(x)

tensor([[-1.2208,  3.2020, -4.1693,  6.1506],
        [-0.2051,  1.1085, -1.0807,  1.9842],
        [-0.3237,  1.9432, -1.8349,  3.4544]], grad_fn=<SqueezeBackward1>)

If we change the `out_channels` we control the number of rows of the output tensor

In [7]:
conv = torch.nn.Conv1d(in_channels = 3, out_channels = 1, kernel_size = 1)
conv(x)

tensor([[-0.3867,  1.2341, -1.4672,  2.3147]], grad_fn=<SqueezeBackward1>)

Now let's change the input tensor just a little bit at `x[1,0]`

In [8]:
x = torch.tensor([[1., -2., 3., -4.],
                  [5., -2., 3., -4.],
                  [1., -2., 3., -4.]])

Now run the same `Conv1d()` over this tensor. 

(Note that we have not re-initialized the Conv1d function so it is the same Conv1d instance as above.)

In [9]:
conv(x)

tensor([[-0.6989,  1.2341, -1.4672,  2.3147]], grad_fn=<SqueezeBackward1>)

Notice that only the first item of the output tensor has changed. So Conv1d processes the tensor "column-by-column" with an expectation of how many rows/dimensions are in each column based on value set to `in_channels`.


### Kernel Size Changed

But now let's run everything again with a different kernel size.

In [10]:
x = torch.tensor([[1., -2., 3., -4.],
                  [1., -2., 3., -4.],
                  [1., -2., 3., -4.]])

In [11]:
conv = torch.nn.Conv1d(in_channels = 3, out_channels = 1, kernel_size = 2)
conv(x)

tensor([[ 1.4349, -1.6519,  3.0375]], grad_fn=<SqueezeBackward1>)

Note the size/shape of the output has changed from (1,4) to (1,3).

 1. The first item of the output is processed from the first and second columns of x
 2. Second item processed from second and third.
 3. Third item processed from third and fourth.

This is because the default `stride` value of `Conv1d()` is set equal to 1. `kernel_size` determines the size of the window that passes over the tensor. `stride` controls how the window passes over.

In [12]:
x = torch.tensor([[1., -2., 3., -4.],
                  [5., -2., 3., -4.],
                  [1., -2., 3., -4.]])

In [13]:
conv(x)

tensor([[ 2.1374, -1.6519,  3.0375]], grad_fn=<SqueezeBackward1>)

Again only the first value has changed. But now let's change numbers in the second column of x

In [14]:
x = torch.tensor([[1., -2., 3., -4.],
                  [1., 2.5, 3., -4.],
                  [1., -2., 3., -4.]])

In [15]:
conv(x)

tensor([[ 0.8089, -0.8616,  3.0375]], grad_fn=<SqueezeBackward1>)

Now the first and second values are different. Since both the first and second values are dependent on values in the second column of x (i.e., the kernel for the first and second outputs both include the second column) this makes sense.

### Stride Changed

In [16]:
x = torch.tensor([[1., -2., 3., -4.],
                  [1., -2., 3., -4.],
                  [1., -2., 3., -4.]])

In [17]:
conv = torch.nn.Conv1d(in_channels = 3, out_channels = 1, kernel_size = 2, stride =2)
conv(x)

tensor([[-1.2416, -2.6543]], grad_fn=<SqueezeBackward1>)

Notice that changing stride affects the size of the output.

In [18]:
x = torch.tensor([[1., -2., 3., -4.],
                  [1., 2.5, 3., -4.],
                  [1., -2., 3., -4.]])

In [19]:
conv(x)

tensor([[ 0.3102, -2.6543]], grad_fn=<SqueezeBackward1>)

Now the first output value depends on x's first and second columns' values, and second output depends on x's third and fourth columns' values. So changing the second column's values only affects the first output value.

## 2D Convolutions

So wait a minute...a 1D convolution can still "handle" a 2-dimensional input or even higher. The "1D" in 1D convolutions doesn't refer to the the 1D size of the input, but the 1D size of the kernel.

Let's try everything again with `Conv2d`

In [20]:
# Notice the tensor is now 3D instead of 2D.
x = torch.tensor([[[1., -2., 3., -4.],
                  [1., -2., 3., -4.],
                  [1., -2., 3., -4.]]])

In [21]:
x.shape

torch.Size([1, 3, 4])

In [22]:
# NOTE: this won't work! in_channels has to equal 1, the first dimension of torch.Size. Think of this as a grayscale image. in_channels = 3 for RGB images. 
# conv = torch.nn.Conv2d(in_channels = 3, out_channels = 1, kernel_size = 2)

In [23]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 2)

In [24]:
conv(x)

tensor([[[0.4745, 0.1738, 0.6863],
         [0.4745, 0.1738, 0.6863]]], grad_fn=<SqueezeBackward1>)

In [25]:
conv(x).shape

torch.Size([1, 2, 3])

Now let's change `out_channels`

In [26]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 2, kernel_size = 2)

In [27]:
conv(x)

tensor([[[-1.2848,  1.1525, -2.1176],
         [-1.2848,  1.1525, -2.1176]],

        [[ 0.5484, -1.5114,  1.2023],
         [ 0.5484, -1.5114,  1.2023]]], grad_fn=<SqueezeBackward1>)

### Kernel Size Change

In [28]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = (2,3))

In [29]:
conv(x)

tensor([[[-0.2927, -0.1780],
         [-0.2927, -0.1780]]], grad_fn=<SqueezeBackward1>)

In [30]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = (3,3))

In [31]:
conv(x)

tensor([[[-2.8365,  3.3714]]], grad_fn=<SqueezeBackward1>)

At the risk of confusing things, you can say that a Conv1D for a 2D tensor of size (R,C) with `kernel_size` = X is the same as a Conv2D where `kernel_size` = (R,X) for a 3D tensor of size (1, R, C) for any value of X

### Stride Changed

In [32]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = (3,3), stride=2)

In [33]:
conv(x)

tensor([[[-0.3545]]], grad_fn=<SqueezeBackward1>)

Notice that when the kernel "window goes outside the tensor" it doesn't break, but just doesn't return any values

In [34]:
# Notice the tensor is now 3D instead of 2D.
x = torch.tensor([[[1., -2., 3.],
                  [1., -2., 3.],
                  [1., -2., 3.]]])

In [35]:
conv(x)

tensor([[[-0.3545]]], grad_fn=<SqueezeBackward1>)

Given the (3,3) kernel, this means that the fourth column's values essentially don't matter to the output of the 2D convolution

### Padding Changed

Start with original tensor and a (3,3) kernel convolution

In [36]:
# Notice the tensor is now 3D instead of 2D.
x = torch.tensor([[[1., -2., 3., -4.],
                  [1., -2., 3., -4.],
                  [1., -2., 3., -4.]]])

In [37]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3)

In [38]:
conv(x)

tensor([[[-1.4029,  2.0937]]], grad_fn=<SqueezeBackward1>)

Change padding from 0 to 1. This adds 1 extra cell on the "border" of the tensor. Each cell is filled with value of 0 (`padding_mode='zeros'`)

In [39]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 3, padding=1)

In [40]:
conv(x)

tensor([[[-0.3422, -0.1828, -0.3376, -0.8155],
         [-0.0103, -0.6718,  0.3085, -1.3115],
         [ 0.2523, -1.0040,  0.7104, -0.6226]]], grad_fn=<SqueezeBackward1>)

### Dilation

Start with original tensor and a (2,2) kernel convolution

NOTE: Default `dilation=1` not 0!!!!

In [41]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 2)

In [42]:
conv(x)

tensor([[[-0.3002,  0.6701, -0.7477],
         [-0.3002,  0.6701, -0.7477]]], grad_fn=<SqueezeBackward1>)

In [43]:
conv = torch.nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = 2, dilation=2)

In [44]:
conv(x)

tensor([[[0.6097, 0.4791]]], grad_fn=<SqueezeBackward1>)

See "Dilated convolution animations" in https://github.com/vdumoulin/conv_arithmetic/blob/master/README.md for visualization of dilations

## Conv3D

In [61]:
x = torch.rand((7,100,100,100))
x.to(device)

tensor([[[[9.7462e-01, 6.8925e-01, 1.9459e-01,  ..., 3.4456e-01,
           2.7675e-01, 1.5181e-01],
          [3.7647e-01, 1.1397e-01, 7.8948e-01,  ..., 9.0990e-01,
           8.4903e-01, 3.7861e-01],
          [8.3438e-02, 9.6676e-01, 3.5842e-01,  ..., 2.4440e-01,
           6.3722e-01, 6.2185e-01],
          ...,
          [7.1625e-01, 5.1116e-03, 7.3384e-02,  ..., 7.7449e-02,
           4.3133e-01, 9.4047e-01],
          [7.5294e-01, 4.8695e-01, 9.0065e-01,  ..., 9.4242e-02,
           2.7371e-01, 5.0946e-01],
          [5.3643e-01, 4.9829e-01, 2.0844e-01,  ..., 8.9906e-01,
           5.6480e-02, 8.0783e-01]],

         [[8.1283e-01, 9.8247e-01, 3.1961e-01,  ..., 4.8675e-01,
           1.4723e-02, 5.7070e-01],
          [2.7571e-01, 1.4831e-01, 6.3304e-02,  ..., 1.7092e-01,
           4.2705e-01, 4.2054e-01],
          [7.5930e-01, 2.9787e-01, 7.0632e-01,  ..., 3.6400e-01,
           2.8239e-01, 4.8157e-01],
          ...,
          [6.1940e-01, 8.7853e-01, 9.4755e-03,  ..., 6.0697

In [62]:
x = x.unsqueeze(0)
x.shape

torch.Size([1, 7, 100, 100, 100])

In [63]:
(N, C, D, H, W) = x.shape

See Shape section in: https://pytorch.org/docs/stable/generated/torch.nn.Conv3d.html

In [ ]:
# D_out = ((D + 2*padding - dilation*(kernel_size-1) - 1)/stride) + 1
# H_out = ((H + 2*padding - dilation*(kernel_size-1) - 1)/stride) + 1
# W_out = ((W + 2*padding - dilation*(kernel_size-1) - 1)/stride) + 1

In [64]:
conv = nn.Conv3d(in_channels=C, out_channels=1, kernel_size=10)
conv(x).shape # 0.6s

torch.Size([1, 1, 91, 91, 91])

In [65]:
conv = nn.Conv3d(in_channels=C, out_channels=1, kernel_size=50)
conv(x).shape # 22.9s

torch.Size([1, 1, 51, 51, 51])

In [66]:
conv = nn.Conv3d(in_channels=C, out_channels=1, kernel_size=10, stride=10)
conv(x).shape # 0.0s

torch.Size([1, 1, 10, 10, 10])

### Larger Input Tensor

Note that code has been commented out because of their long runtimes.

In [ ]:
x = torch.rand((7,500,500,500))

In [47]:
x = x.unsqueeze(0)
x.shape

torch.Size([1, 7, 500, 500, 500])

In [48]:
(N, C, D, H, W) = x.shape

In [49]:
x.to(device)

tensor([[[[[5.0772e-01, 2.8245e-01, 5.0260e-01,  ..., 2.0511e-02,
            3.1671e-01, 2.2472e-01],
           [6.4318e-01, 3.9108e-02, 8.8381e-01,  ..., 9.5520e-02,
            7.2430e-01, 8.0891e-02],
           [2.3047e-01, 9.4681e-01, 6.8016e-01,  ..., 2.5511e-02,
            5.4949e-01, 8.7874e-01],
           ...,
           [7.9657e-01, 2.6782e-01, 6.7528e-01,  ..., 5.2497e-01,
            5.6398e-01, 9.9298e-02],
           [8.3544e-01, 5.7781e-01, 5.6891e-01,  ..., 6.5298e-01,
            2.9631e-01, 7.5155e-01],
           [7.5463e-01, 7.8484e-01, 1.2507e-01,  ..., 9.3057e-01,
            1.6868e-01, 1.1122e-01]],

          [[7.2107e-01, 5.2380e-01, 9.3793e-01,  ..., 9.0589e-01,
            5.8841e-02, 3.0431e-01],
           [8.2370e-01, 7.1802e-01, 8.2395e-01,  ..., 3.4387e-02,
            8.3237e-01, 6.0437e-01],
           [4.2817e-01, 2.1229e-01, 1.7281e-01,  ..., 5.8156e-01,
            7.2536e-01, 1.5066e-01],
           ...,
           [1.8736e-01, 9.5154e-01, 1.3

In [50]:
conv = nn.Conv3d(in_channels=C, out_channels=1, kernel_size=10)

In [51]:
# x - torch.Size([1, 7, 500, 500, 500])

# conv(x).shape # torch.Size([1, 1, 491, 491, 491])

# and this takes ~2min to complete

In [52]:
conv = nn.Conv3d(in_channels=C, out_channels=1, kernel_size=100)

In [53]:
# x - torch.Size([1, 7, 500, 500, 500])

# conv(x).shape 

# takes more than 8 min!!!

In [54]:
conv = nn.Conv3d(in_channels=C, out_channels=1, kernel_size=100, stride=50)

In [55]:
conv(x).shape # torch.Size([1, 1, 9, 9, 9])

# takes 2 seconds!

torch.Size([1, 1, 9, 9, 9])

In [56]:
conv = nn.Conv3d(in_channels=C, out_channels=1, kernel_size=100, stride=10)

In [57]:
# conv(x).shape # torch.Size([1, 1, 41, 41, 41])

# and this takes ~2min to complete

torch.Size([1, 1, 41, 41, 41])